<a href="https://colab.research.google.com/github/Wiki626/PsuedoCyintists-698S-Final-Project/blob/master/Times%20Webscrapper%20Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Import required libs
from bs4 import BeautifulSoup as bs
import requests
import pickle

# Set keywords and the first site to search
base_url = 'https://timesofindia.indiatimes.com'
location = '/world/us'

keywords = ('Fed', 'White House', 'Trump', 'US Senate', 'US Government', 'Supreme Court', 'House of Representatives',
            'Congress', 'US President', 'Capitol Hill', 'Washington')

# Function for setting up the soup
def get_soup(base_url, location):
    page = requests.get(base_url + location)
    soup = bs(page.content, 'html.parser')
    return soup


soup = get_soup(base_url, location)

# Grabs the "top news" stories from the page and iterates down to the "li" tags
top_news = {}
top_news_list = soup.find('ul', class_='top-newslist clearfix')
top_news_items = top_news_list.find_all('li')

# Grabs the html location and title for each "top news" story and saves them to a dictionary
for item in top_news_items:
    for top_news_items in item.find_all('a'):
        top_news.update({top_news_items.get('href'): top_news_items.get('title')})

# Cleans the dictionary of "None" type entries
filtered = {k: v for k, v in top_news.items() if v is not None}
top_news.clear()
top_news.update(filtered)

# Grabs the "latest news" stories from the page and iterates down to the "li" tags
latest_stories = {}
latest_stories_list = soup.find('ul', class_='list5 clearfix')
latest_stories_items = latest_stories_list.find_all('li')

# Grabs the html location and title for each "latest news" story and saves them to a dictionary
for item in latest_stories_items:
    for latest_stories_items in item.find_all('a'):
        latest_stories.update({latest_stories_items.get('href'): latest_stories_items.get('title')})

# Cleans the dictionary of "None" type entries
filtered = {k: v for k, v in latest_stories.items() if v is not None}
latest_stories.clear()
latest_stories.update(filtered)

# Definition for filtering the values of dictionaries by keywords and return keys to a list (only if the key has not
# already been added to that list)
def dict_search(input_dict, keywords_list, save_location):
    temp = []
    for k, v in input_dict.items():
        for keyword in keywords_list:
            if keyword in v:
                if v not in save_location:
                    save_location.append(k)
                    temp.append(keyword)
    print(f'The keywords that triggered were {temp}.')


top_news_links = []
latest_stories_links = []

# Pulling the html location keys out of the dictionaries created above into a pair of lists
dict_search(top_news, keywords, top_news_links)
print(f'{len(top_news_links)} results where found in Times of India US Top News Stories.')
dict_search(latest_stories, keywords, latest_stories_links)
print(f'{len(latest_stories_links)} results where found in Times of India US Latest News Stories.')

article_text = ''

# Definition for pulling the body text from the Time of India articles identified above
def article_lookup(links, base_url):
    for link in links:
        soup = get_soup(base_url, link)
        temp = soup.find(class_='_3WlLe')
        global article_text
        article_text = article_text + ' ' + temp.get_text()

# Pulling the articles and saving their text to a string
article_lookup(top_news_links, base_url)
article_lookup(latest_stories_links, base_url)

print(f'The results are: {article_text}')

pickle.dump(article_text, open('times_article_text.p', 'wb'))

# Import libs
from textblob import TextBlob as tb
import numpy as np
import nltk

# Pull in pickles
times_article_text = pickle.load(open('times_article_text.p', 'rb'))

# Create blobs
times_blob = tb(times_article_text)
times_blob_sentences = times_blob.sentences

filtered_times_blob_sentences = []

# Clear out advertising
for sentence in reversed(times_blob_sentences):
    dirty_words = 'Download The Times of India News App for Latest World News.'
    if dirty_words not in sentence:
        filtered_times_blob_sentences.append(sentence)

times_sentiment = []

# Definition for determining sentiment per sentence
def sentence_sentiment(sentence_blob, storage_location):
    for sentence in sentence_blob:
        sentiment = sentence.sentiment[0]*sentence.sentiment[1]
        storage_location.append(sentiment)
        #print(f'The sentiment score is {sentiment} for "{sentence}"')

# Getting the average sentiment for a site
sentence_sentiment(filtered_times_blob_sentences, times_sentiment)
times_ave_sentiment = np.average(times_sentiment)
print(f'The average sentiment for The Times of India is {times_ave_sentiment}')

The keywords that triggered were ['Trump'].
1 results where found in Times of India US Top News Stories.
The keywords that triggered were ['Fed', 'Trump', 'White House', 'Trump', 'Trump', 'Trump', 'Fed', 'Trump', 'Trump', 'Trump', 'US President', 'Trump', 'White House', 'Trump', 'US President', 'Trump', 'Trump', 'US President', 'Trump', 'Trump', 'Trump', 'Trump', 'Trump', 'Trump', 'Trump', 'Trump'].
26 results where found in Times of India US Latest News Stories.
The results are:  WASHINGTON: US President Donald Trump said Monday postponing elections over the coronavirus crisis was "unnecessary," even as Ohio's governor called for the state's presidential primary vote scheduled for Tuesday to be delayed. "Postponing elections is not a very good thing," Trump told reporters at the White House. "I think postponing is unnecessary." Three other states voting Tuesday -- Arizona, Florida and Illinois -- are scheduled to hold primaries, which select party candidates to contest the presidentia